In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# AutoML SDK: AutoML image classification model




## Installation

Install the latest (preview) version of AutoML SDK.



In [ ]:
! pip3 install -U google-cloud-automl --user


Install the Google *cloud-storage* library as well.



In [ ]:
! pip3 install google-cloud-storage


### Restart the Kernel

Once you've installed the AutoML SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.



In [ ]:
import os


if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AutoML APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AutoML Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.



In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)


In [ ]:
! gcloud config set project $PROJECT_ID


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AutoML. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AutoML. Not all regions provide support for all AutoML services. For the latest support per region, see [Region support for AutoML services]()



In [ ]:
REGION = 'us-central1' #@param {type: "string"}


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.



In [ ]:
from datetime import datetime


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


### Authenticate your GCP account

**If you are using AutoML Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AutoML notebook and run the cell, the cell knows to skip executing the authentication steps.*



In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AutoML, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.



In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}


In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.



In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME


Finally, validate access to your Cloud Storage bucket by examining its contents:



In [ ]:
! gsutil ls -al gs://$BUCKET_NAME


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants



#### Import AutoML SDK

Import the AutoML SDK into our Python environment.



In [ ]:
import json
import os
import sys
import time


from google.cloud import automl


from google.protobuf.json_format import MessageToJson
from google.protobuf.json_format import ParseDict
from googleapiclient.discovery import build


#### AutoML constants

Setup up the following constants for AutoML:

- `PARENT`: The AutoML location root path for dataset, model and endpoint resources.



In [ ]:
# AutoML location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION


## Clients

The AutoML SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AutoML).

You will use several clients in this tutorial, so set them all up upfront.



In [ ]:
def automl_client():
    return automl.AutoMlClient()


def prediction_client():
    return automl.PredictionServiceClient()


def operations_client():
    return automl.AutoMlClient()._transport.operations_client


clients = {}
clients["automl"] = automl_client()
clients["prediction"] = prediction_client()
clients["operations"] = operations_client()

for client in clients.items():
    print(client)


In [ ]:
IMPORT_FILE =  "gs://cloud-ml-data/img/flower_photos/train_set.csv"


In [ ]:
#%%capture
! gsutil cp -r gs://cloud-ml-data/img/flower_photos/ gs://$BUCKET_NAME


In [ ]:
import tensorflow as tf

all_files_csv = ! gsutil cat $IMPORT_FILE
all_files_csv = [ l.replace("cloud-ml-data/img", BUCKET_NAME) for l in all_files_csv ]

IMPORT_FILE = "gs://" + BUCKET_NAME   + "/flower_photos/train_set.csv"
with tf.io.gfile.GFile(IMPORT_FILE, 'w') as f:
    for l in all_files_csv:
        f.write(l + "\n")


In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10

*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/daisy/754296579_30a9ae018c_n.jpg,daisy
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg,dandelion
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/dandelion/284497199_93a01f48f6.jpg,dandelion
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/dandelion/3554992110_81d8c9b0bd_m.jpg,dandelion
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/daisy/4065883015_4bb6010cb7_n.jpg,daisy
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/roses/7420699022_60fa574524_m.jpg,roses
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/dandelion/4558536575_d43a611bd4_n.jpg,dandelion
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/daisy/7568630428_8cf0fc16ff_n.jpg,daisy
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/tulips/7064813645_f7f48fb527.jpg,tulips
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/sunflowers/4933229095_f7e4218b28.jpg,sunflowers
```



## Create a dataset



### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request



In [ ]:
dataset = {
    "display_name": "flowers_" + TIMESTAMP,
    "image_classification_dataset_metadata": {
        "classification_type": "MULTICLASS",
    },
}

print(MessageToJson(
    automl.CreateDatasetRequest(
        parent=PARENT,
        dataset=dataset,
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "flowers_20210226015151",
    "imageClassificationDatasetMetadata": {
      "classificationType": "MULTICLASS"
    }
  }
}
```



#### Call



In [ ]:
request = clients["automl"].create_dataset(
    parent=PARENT,
    dataset=dataset,
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/ICN2833688305139187712"
}
```



In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)



### [projects.locations.datasets.importData](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/importData)

#### Request



In [ ]:
input_config = {
    "gcs_source": {
        "input_uris": [IMPORT_FILE],
    },
}

print(MessageToJson(
    automl.ImportDataRequest(
        name=dataset_short_id,
        input_config=input_config
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "name": "ICN2833688305139187712",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210226015151/flower_photos/train_set.csv"
      ]
    }
  }
}
```



#### Call



In [ ]:
request = clients["automl"].import_data(
    name=dataset_id,
    input_config=input_config
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:
```
{}
```



## Train a model



### [projects.locations.models.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

#### Request



In [ ]:
model = {
    "display_name": "flowers_" + TIMESTAMP,
    "dataset_id": dataset_short_id,
    "image_classification_model_metadata": {
        "train_budget_milli_node_hours": 8000,
    },
}

print(MessageToJson(
    automl.CreateModelRequest(
        parent=PARENT,
        model=model,
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "flowers_20210226015151",
    "datasetId": "ICN2833688305139187712",
    "imageClassificationModelMetadata": {
      "trainBudgetMilliNodeHours": "8000"
    }
  }
}
```



#### Call



In [ ]:
request = clients["automl"].create_model(
    parent=PARENT,
    model=model,
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168"
}
```



In [ ]:
# The full unique ID for the training pipeline
model_id = result.name
# The short numeric ID for the training pipeline
model_short_id = model_id.split('/')[-1]

print(model_short_id)


## Evaluate the model



### [projects.locations.models.modelEvaluations.list](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/list)

#### Call



In [ ]:
request = clients["automl"].list_model_evaluations(
    parent=model_id, 
)


#### Response



In [ ]:
import json


model_evaluations = [
    json.loads(MessageToJson(me.__dict__["_pb"])) for me in request 
]
# The evaluation slice
evaluation_slice = request.model_evaluation[0].name

print(json.dumps(model_evaluations, indent=2))


*Example output*
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168/modelEvaluations/1701367336556072668",
    "createTime": "2021-02-26T03:00:19.383521Z",
    "evaluatedExampleCount": 329,
    "classificationEvaluationMetrics": {
      "auPrc": 0.99747145,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.2
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 0.99088144,
          "precision": 0.92877495
        },
        {
          "confidenceThreshold": 0.1,
          "recall": 0.98784196,
          "precision": 0.9447674
        },
        {
          "confidenceThreshold": 0.15,
          "recall": 0.9848024,
          "precision": 0.9501466
        },
        {
          "confidenceThreshold": 0.2,
          "recall": 0.9848024,
          "precision": 0.96142435
        },
        {
          "confidenceThreshold": 0.25,
          "recall": 0.98176295,
          "precision": 0.9641791
        },
        {
          "confidenceThreshold": 0.3,
          "recall": 0.98176295,
          "precision": 0.9670659
        },
        {
          "confidenceThreshold": 0.35,
          "recall": 0.9787234,
          "precision": 0.966967
        },
        {
          "confidenceThreshold": 0.4,
          "recall": 0.97568387,
          "precision": 0.96686745
        },
        {
          "confidenceThreshold": 0.45,
          "recall": 0.97568387,
          "precision": 0.9727273
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.9726444,
          "precision": 0.9756098
        },
        {
          "confidenceThreshold": 0.55,
          "recall": 0.9726444,
          "precision": 0.9756098
        },
        {
          "confidenceThreshold": 0.6,
          "recall": 0.9665654,
          "precision": 0.9754601
        },
        {
          "confidenceThreshold": 0.65,
          "recall": 0.9665654,
          "precision": 0.9814815
        },
        {
          "confidenceThreshold": 0.7,
          "recall": 0.9665654,
          "precision": 0.98452014
        },
        {
          "confidenceThreshold": 0.75,
          "recall": 0.9665654,
          "precision": 0.98452014
        },
        {
          "confidenceThreshold": 0.8,
          "recall": 0.9604863,
          "precision": 0.9875
        },
        {
          "confidenceThreshold": 0.85,
          "recall": 0.9452888,
          "precision": 0.99044585
        },
        {
          "confidenceThreshold": 0.875,
          "recall": 0.94224924,
          "precision": 0.99041533
        },
        {
          "confidenceThreshold": 0.9,
          "recall": 0.9392097,
          "precision": 0.99038464
        },
        {
          "confidenceThreshold": 0.91,
          "recall": 0.9392097,
          "precision": 0.99038464
        },
        {
          "confidenceThreshold": 0.92,
          "recall": 0.9361702,
          "precision": 0.9935484
        },
        {
          "confidenceThreshold": 0.93,
          "recall": 0.9361702,
          "precision": 0.9935484
        },
        {
          "confidenceThreshold": 0.94,
          "recall": 0.9361702,
          "precision": 0.9935484
        },
        {
          "confidenceThreshold": 0.95,
          "recall": 0.9331307,
          "precision": 0.99352753
        },
        {
          "confidenceThreshold": 0.96,
          "recall": 0.9300912,
          "precision": 0.99674267
        },
        {
          "confidenceThreshold": 0.97,
          "recall": 0.92705166,
          "precision": 0.996732
        },
        {
          "confidenceThreshold": 0.98,
          "recall": 0.9148936,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.99,
          "recall": 0.89361703,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.995,
          "recall": 0.88145894,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.996,
          "recall": 0.87234044,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.997,
          "recall": 0.8693009,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.998,
          "recall": 0.8449848,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.999,
          "recall": 0.81155014,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 1.0,
          "recall": 0.24012157,
          "precision": 1.0
        }
      ],
      "confusionMatrix": {
        "annotationSpecId": [
          "548545251585818624",
          "4295540141558071296",
          "5160231270013206528",
          "6601383150771765248",
          "8907226159985459200"
        ],
        "row": [
          {
            "exampleCount": [
              55,
              0,
              1,
              2,
              0
            ]
          },
          {
            "exampleCount": [
              0,
              59,
              1,
              0,
              1
            ]
          },
          {
            "exampleCount": [
              0,
              0,
              81,
              0,
              0
            ]
          },
          {
            "exampleCount": [
              0,
              0,
              0,
              73,
              0
            ]
          },
          {
            "exampleCount": [
              0,
              1,
              2,
              0,
              53
            ]
          }
        ],
        "displayName": [
          "roses",
          "sunflowers",
          "dandelion",
          "tulips",
          "daisy"
        ]
      },
      "logLoss": 0.02853713
    }
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168/modelEvaluations/4464795143994212237",
    "annotationSpecId": "6601383150771765248",
    "createTime": "2021-02-26T03:00:19.383521Z",
    "classificationEvaluationMetrics": {
      "auPrc": 0.9990742,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.2218845
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 1.0,
          "precision": 0.8795181
        },
        {
          "confidenceThreshold": 0.1,
          "recall": 1.0,
          "precision": 0.9125
        },
        {
          "confidenceThreshold": 0.15,
          "recall": 1.0,
          "precision": 0.9240506
        },
        {
          "confidenceThreshold": 0.2,
          "recall": 1.0,
          "precision": 0.9605263
        },
        {
          "confidenceThreshold": 0.25,
          "recall": 1.0,
          "precision": 0.9605263
        },
        {
          "confidenceThreshold": 0.3,
          "recall": 1.0,
          "precision": 0.97333336
        },
        {
          "confidenceThreshold": 0.35,
          "recall": 1.0,
          "precision": 0.97333336
        },
        {
          "confidenceThreshold": 0.4,
          "recall": 1.0,
          "precision": 0.97333336
        },
        {
          "confidenceThreshold": 0.45,
          "recall": 1.0,
          "precision": 0.97333336
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.98630136,
          "precision": 0.972973
        },
        {
          "confidenceThreshold": 0.55,
          "recall": 0.98630136,
          "precision": 0.972973
        },
        {
          "confidenceThreshold": 0.6,
          "recall": 0.9726027,
          "precision": 0.9726027
        },
        {
          "confidenceThreshold": 0.65,
          "recall": 0.9726027,
          "precision": 0.9726027
        },
        {
          "confidenceThreshold": 0.7,
          "recall": 0.9726027,
          "precision": 0.9726027
        },
        {
          "confidenceThreshold": 0.75,
          "recall": 0.9726027,
          "precision": 0.9726027
        },
        {
          "confidenceThreshold": 0.8,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.85,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.875,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.9,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.91,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.92,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.93,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.94,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.95,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.96,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.97,
          "recall": 0.9726027,
          "precision": 0.9861111
        },
        {
          "confidenceThreshold": 0.98,
          "recall": 0.9589041,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.99,
          "recall": 0.9315069,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.995,
          "recall": 0.91780823,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.996,
          "recall": 0.91780823,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.997,
          "recall": 0.91780823,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.998,
          "recall": 0.9041096,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.999,
          "recall": 0.8356164,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 1.0,
          "recall": 0.12328767,
          "precision": 1.0
        }
      ],
      "auRoc": 0.99973243,
      "logLoss": 0.024023052
    },
    "displayName": "tulips"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168/modelEvaluations/6132683338167493052",
    "annotationSpecId": "8907226159985459200",
    "createTime": "2021-02-26T03:00:19.383521Z",
    "classificationEvaluationMetrics": {
      "auPrc": 0.99841,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.17021276
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 1.0,
          "precision": 0.9655172
        },
        {
          "confidenceThreshold": 0.1,
          "recall": 0.98214287,
          "precision": 0.9649123
        },
        {
          "confidenceThreshold": 0.15,
          "recall": 0.98214287,
          "precision": 0.98214287
        },
        {
          "confidenceThreshold": 0.2,
          "recall": 0.98214287,
          "precision": 0.98214287
        },
        {
          "confidenceThreshold": 0.25,
          "recall": 0.98214287,
          "precision": 0.98214287
        },
        {
          "confidenceThreshold": 0.3,
          "recall": 0.98214287,
          "precision": 0.98214287
        },
        {
          "confidenceThreshold": 0.35,
          "recall": 0.96428573,
          "precision": 0.9818182
        },
        {
          "confidenceThreshold": 0.4,
          "recall": 0.9464286,
          "precision": 0.9814815
        },
        {
          "confidenceThreshold": 0.45,
          "recall": 0.9464286,
          "precision": 0.9814815
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.9464286,
          "precision": 0.9814815
        },
        {
          "confidenceThreshold": 0.55,
          "recall": 0.9464286,
          "precision": 0.9814815
        },
        {
          "confidenceThreshold": 0.6,
          "recall": 0.9285714,
          "precision": 0.9811321
        },
        {
          "confidenceThreshold": 0.65,
          "recall": 0.9285714,
          "precision": 0.9811321
        },
        {
          "confidenceThreshold": 0.7,
          "recall": 0.9285714,
          "precision": 0.9811321
        },
        {
          "confidenceThreshold": 0.75,
          "recall": 0.9285714,
          "precision": 0.9811321
        },
        {
          "confidenceThreshold": 0.8,
          "recall": 0.9285714,
          "precision": 0.9811321
        },
        {
          "confidenceThreshold": 0.85,
          "recall": 0.9285714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.875,
          "recall": 0.9285714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.9,
          "recall": 0.9285714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.91,
          "recall": 0.9285714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.92,
          "recall": 0.9285714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.93,
          "recall": 0.9285714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.94,
          "recall": 0.9285714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.95,
          "recall": 0.91071427,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.96,
          "recall": 0.91071427,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.97,
          "recall": 0.91071427,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.98,
          "recall": 0.91071427,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.99,
          "recall": 0.875,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.995,
          "recall": 0.83928573,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.996,
          "recall": 0.8214286,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.997,
          "recall": 0.8035714,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.998,
          "recall": 0.78571427,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.999,
          "recall": 0.76785713,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 1.0,
          "recall": 0.30357143,
          "precision": 1.0
        }
      ],
      "auRoc": 0.99967295,
      "logLoss": 0.022124559
    },
    "displayName": "daisy"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168/modelEvaluations/7147485663377408481",
    "annotationSpecId": "548545251585818624",
    "createTime": "2021-02-26T03:00:19.383521Z",
    "classificationEvaluationMetrics": {
      "auPrc": 0.9971625,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.1762918
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 0.9655172,
          "precision": 0.93333334
        },
        {
          "confidenceThreshold": 0.1,
          "recall": 0.9655172,
          "precision": 0.9655172
        },
        {
          "confidenceThreshold": 0.15,
          "recall": 0.9655172,
          "precision": 0.9655172
        },
        {
          "confidenceThreshold": 0.2,
          "recall": 0.9655172,
          "precision": 0.9655172
        },
        {
          "confidenceThreshold": 0.25,
          "recall": 0.94827586,
          "precision": 0.9649123
        },
        {
          "confidenceThreshold": 0.3,
          "recall": 0.94827586,
          "precision": 0.9649123
        },
        {
          "confidenceThreshold": 0.35,
          "recall": 0.94827586,
          "precision": 0.9649123
        },
        {
          "confidenceThreshold": 0.4,
          "recall": 0.94827586,
          "precision": 0.9649123
        },
        {
          "confidenceThreshold": 0.45,
          "recall": 0.94827586,
          "precision": 0.98214287
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.94827586,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.55,
          "recall": 0.94827586,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.6,
          "recall": 0.94827586,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.65,
          "recall": 0.94827586,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.7,
          "recall": 0.94827586,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.75,
          "recall": 0.94827586,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.8,
          "recall": 0.94827586,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.85,
          "recall": 0.87931037,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.875,
          "recall": 0.87931037,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.9,
          "recall": 0.87931037,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.91,
          "recall": 0.87931037,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.92,
          "recall": 0.86206895,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.93,
          "recall": 0.86206895,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.94,
          "recall": 0.86206895,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.95,
          "recall": 0.86206895,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.96,
          "recall": 0.86206895,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.97,
          "recall": 0.86206895,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.98,
          "recall": 0.8448276,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.99,
          "recall": 0.79310346,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.995,
          "recall": 0.79310346,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.996,
          "recall": 0.7758621,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.997,
          "recall": 0.7758621,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.998,
          "recall": 0.70689654,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.999,
          "recall": 0.6896552,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 1.0,
          "recall": 0.03448276,
          "precision": 1.0
        }
      ],
      "auRoc": 0.9993638,
      "logLoss": 0.034111425
    },
    "displayName": "roses"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168/modelEvaluations/8076647367053688867",
    "annotationSpecId": "5160231270013206528",
    "createTime": "2021-02-26T03:00:19.383521Z",
    "classificationEvaluationMetrics": {
      "auPrc": 0.9989403,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.2462006
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 1.0,
          "precision": 0.9101124
        },
        {
          "confidenceThreshold": 0.1,
          "recall": 1.0,
          "precision": 0.92045456
        },
        {
          "confidenceThreshold": 0.15,
          "recall": 1.0,
          "precision": 0.92045456
        },
        {
          "confidenceThreshold": 0.2,
          "recall": 1.0,
          "precision": 0.9310345
        },
        {
          "confidenceThreshold": 0.25,
          "recall": 1.0,
          "precision": 0.94186044
        },
        {
          "confidenceThreshold": 0.3,
          "recall": 1.0,
          "precision": 0.94186044
        },
        {
          "confidenceThreshold": 0.35,
          "recall": 1.0,
          "precision": 0.94186044
        },
        {
          "confidenceThreshold": 0.4,
          "recall": 1.0,
          "precision": 0.94186044
        },
        {
          "confidenceThreshold": 0.45,
          "recall": 1.0,
          "precision": 0.9529412
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 1.0,
          "precision": 0.9529412
        },
        {
          "confidenceThreshold": 0.55,
          "recall": 1.0,
          "precision": 0.9529412
        },
        {
          "confidenceThreshold": 0.6,
          "recall": 1.0,
          "precision": 0.9529412
        },
        {
          "confidenceThreshold": 0.65,
          "recall": 1.0,
          "precision": 0.96428573
        },
        {
          "confidenceThreshold": 0.7,
          "recall": 1.0,
          "precision": 0.97590363
        },
        {
          "confidenceThreshold": 0.75,
          "recall": 1.0,
          "precision": 0.97590363
        },
        {
          "confidenceThreshold": 0.8,
          "recall": 0.9876543,
          "precision": 0.9756098
        },
        {
          "confidenceThreshold": 0.85,
          "recall": 0.9876543,
          "precision": 0.9756098
        },
        {
          "confidenceThreshold": 0.875,
          "recall": 0.97530866,
          "precision": 0.97530866
        },
        {
          "confidenceThreshold": 0.9,
          "recall": 0.962963,
          "precision": 0.975
        },
        {
          "confidenceThreshold": 0.91,
          "recall": 0.962963,
          "precision": 0.975
        },
        {
          "confidenceThreshold": 0.92,
          "recall": 0.962963,
          "precision": 0.98734176
        },
        {
          "confidenceThreshold": 0.93,
          "recall": 0.962963,
          "precision": 0.98734176
        },
        {
          "confidenceThreshold": 0.94,
          "recall": 0.962963,
          "precision": 0.98734176
        },
        {
          "confidenceThreshold": 0.95,
          "recall": 0.962963,
          "precision": 0.98734176
        },
        {
          "confidenceThreshold": 0.96,
          "recall": 0.9506173,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.97,
          "recall": 0.9506173,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.98,
          "recall": 0.9506173,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.99,
          "recall": 0.9506173,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.995,
          "recall": 0.9506173,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.996,
          "recall": 0.9506173,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.997,
          "recall": 0.9506173,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.998,
          "recall": 0.9382716,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.999,
          "recall": 0.9259259,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 1.0,
          "recall": 0.5555556,
          "precision": 1.0
        }
      ],
      "auRoc": 0.99965155,
      "logLoss": 0.029262401
    },
    "displayName": "dandelion"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168/modelEvaluations/8816571236383372686",
    "annotationSpecId": "4295540141558071296",
    "createTime": "2021-02-26T03:00:19.383521Z",
    "classificationEvaluationMetrics": {
      "auPrc": 0.99703646,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.18541034
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 0.9836066,
          "precision": 0.9836066
        },
        {
          "confidenceThreshold": 0.1,
          "recall": 0.9836066,
          "precision": 0.9836066
        },
        {
          "confidenceThreshold": 0.15,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.2,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.25,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.3,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.35,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.4,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.45,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.55,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.6,
          "recall": 0.9672131,
          "precision": 0.98333335
        },
        {
          "confidenceThreshold": 0.65,
          "recall": 0.9672131,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.7,
          "recall": 0.9672131,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.75,
          "recall": 0.9672131,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.8,
          "recall": 0.9508197,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.85,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.875,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.9,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.91,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.92,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.93,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.94,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.95,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.96,
          "recall": 0.93442625,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.97,
          "recall": 0.91803277,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.98,
          "recall": 0.8852459,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.99,
          "recall": 0.8852459,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.995,
          "recall": 0.86885244,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.996,
          "recall": 0.852459,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.997,
          "recall": 0.852459,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.998,
          "recall": 0.8360656,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.999,
          "recall": 0.78688526,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 1.0,
          "recall": 0.09836066,
          "precision": 1.0
        }
      ],
      "auRoc": 0.9992048,
      "logLoss": 0.03316421
    },
    "displayName": "sunflowers"
  }
]
```



### [projects.locations.models.modelEvaluations.get](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/get)

#### Call



In [ ]:
request = clients["automl"].get_model_evaluation(
    name=evaluation_slice,
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168/modelEvaluations/1701367336556072668",
  "createTime": "2021-02-26T03:00:19.383521Z",
  "evaluatedExampleCount": 329,
  "classificationEvaluationMetrics": {
    "auPrc": 0.99747145,
    "confidenceMetricsEntry": [
      {
        "recall": 1.0,
        "precision": 0.2
      },
      {
        "confidenceThreshold": 0.05,
        "recall": 0.99088144,
        "precision": 0.92877495
      },
      {
        "confidenceThreshold": 0.1,
        "recall": 0.98784196,
        "precision": 0.9447674
      },
      {
        "confidenceThreshold": 0.15,
        "recall": 0.9848024,
        "precision": 0.9501466
      },
      {
        "confidenceThreshold": 0.2,
        "recall": 0.9848024,
        "precision": 0.96142435
      },
      {
        "confidenceThreshold": 0.25,
        "recall": 0.98176295,
        "precision": 0.9641791
      },
      {
        "confidenceThreshold": 0.3,
        "recall": 0.98176295,
        "precision": 0.9670659
      },
      {
        "confidenceThreshold": 0.35,
        "recall": 0.9787234,
        "precision": 0.966967
      },
      {
        "confidenceThreshold": 0.4,
        "recall": 0.97568387,
        "precision": 0.96686745
      },
      {
        "confidenceThreshold": 0.45,
        "recall": 0.97568387,
        "precision": 0.9727273
      },
      {
        "confidenceThreshold": 0.5,
        "recall": 0.9726444,
        "precision": 0.9756098
      },
      {
        "confidenceThreshold": 0.55,
        "recall": 0.9726444,
        "precision": 0.9756098
      },
      {
        "confidenceThreshold": 0.6,
        "recall": 0.9665654,
        "precision": 0.9754601
      },
      {
        "confidenceThreshold": 0.65,
        "recall": 0.9665654,
        "precision": 0.9814815
      },
      {
        "confidenceThreshold": 0.7,
        "recall": 0.9665654,
        "precision": 0.98452014
      },
      {
        "confidenceThreshold": 0.75,
        "recall": 0.9665654,
        "precision": 0.98452014
      },
      {
        "confidenceThreshold": 0.8,
        "recall": 0.9604863,
        "precision": 0.9875
      },
      {
        "confidenceThreshold": 0.85,
        "recall": 0.9452888,
        "precision": 0.99044585
      },
      {
        "confidenceThreshold": 0.875,
        "recall": 0.94224924,
        "precision": 0.99041533
      },
      {
        "confidenceThreshold": 0.9,
        "recall": 0.9392097,
        "precision": 0.99038464
      },
      {
        "confidenceThreshold": 0.91,
        "recall": 0.9392097,
        "precision": 0.99038464
      },
      {
        "confidenceThreshold": 0.92,
        "recall": 0.9361702,
        "precision": 0.9935484
      },
      {
        "confidenceThreshold": 0.93,
        "recall": 0.9361702,
        "precision": 0.9935484
      },
      {
        "confidenceThreshold": 0.94,
        "recall": 0.9361702,
        "precision": 0.9935484
      },
      {
        "confidenceThreshold": 0.95,
        "recall": 0.9331307,
        "precision": 0.99352753
      },
      {
        "confidenceThreshold": 0.96,
        "recall": 0.9300912,
        "precision": 0.99674267
      },
      {
        "confidenceThreshold": 0.97,
        "recall": 0.92705166,
        "precision": 0.996732
      },
      {
        "confidenceThreshold": 0.98,
        "recall": 0.9148936,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 0.99,
        "recall": 0.89361703,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 0.995,
        "recall": 0.88145894,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 0.996,
        "recall": 0.87234044,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 0.997,
        "recall": 0.8693009,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 0.998,
        "recall": 0.8449848,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 0.999,
        "recall": 0.81155014,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 1.0,
        "recall": 0.24012157,
        "precision": 1.0
      }
    ],
    "confusionMatrix": {
      "annotationSpecId": [
        "548545251585818624",
        "4295540141558071296",
        "5160231270013206528",
        "6601383150771765248",
        "8907226159985459200"
      ],
      "row": [
        {
          "exampleCount": [
            55,
            0,
            1,
            2,
            0
          ]
        },
        {
          "exampleCount": [
            0,
            59,
            1,
            0,
            1
          ]
        },
        {
          "exampleCount": [
            0,
            0,
            81,
            0,
            0
          ]
        },
        {
          "exampleCount": [
            0,
            0,
            0,
            73,
            0
          ]
        },
        {
          "exampleCount": [
            0,
            1,
            2,
            0,
            53
          ]
        }
      ],
      "displayName": [
        "roses",
        "sunflowers",
        "dandelion",
        "tulips",
        "daisy"
      ]
    },
    "logLoss": 0.02853713
  }
}
```



## Make batch predictions



### Make a batch prediction file



In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2

if len(str(test_items[0]).split(',')) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(',')
    _, test_item_2, test_label_2 = str(test_items[1]).split(',')
else:
    test_item_1, test_label_1 = str(test_items[0]).split(',')
    test_item_2, test_label_2 = str(test_items[1]).split(',')

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)


*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/daisy/754296579_30a9ae018c_n.jpg daisy
gs://migration-ucaip-trainingaip-20210226015151/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg dandelion
```



In [ ]:
file_1 = test_item_1.split('/')[-1]
file_2 = test_item_2.split('/')[-1]

! gsutil cp $test_item_1 gs://$BUCKET_NAME/$file_1
! gsutil cp $test_item_2 gs://$BUCKET_NAME/$file_2

test_item_1 = "gs://" + BUCKET_NAME + "/" + file_1
test_item_2 = "gs://" + BUCKET_NAME + "/" + file_2


### Make the batch input file



In [ ]:
import tensorflow as tf
import json


gcs_input_uri = "gs://" + BUCKET_NAME + '/test.csv'
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    f.write(test_item_1 + '\n')
    f.write(test_item_2 + '\n')

!gsutil cat $gcs_input_uri


*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015151/754296579_30a9ae018c_n.jpg
gs://migration-ucaip-trainingaip-20210226015151/18089878729_907ed2c7cd_m.jpg
```



### [projects.locations.models.batchPredict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/batchPredict)

#### Request



In [ ]:
input_config = {
    "gcs_source": {
        "input_uris": [gcs_input_uri]
    },
}

output_config = {
    "gcs_destination": {
        "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
    }
}

batch_prediction = automl.BatchPredictRequest(
    name=model_id,
    input_config=input_config,
    output_config=output_config
)

print(MessageToJson(
    batch_prediction.__dict__["_pb"])
)


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210226015151/test.csv"
      ]
    }
  },
  "outputConfig": {
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015151/batch_output/"
    }
  }
}
```



#### Call



In [ ]:
request = clients["prediction"].batch_predict(
    request=batch_prediction
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{}
```



In [ ]:
destination_uri = batch_prediction.output_config.gcs_destination.output_uri_prefix[:-1]

! gsutil ls $destination_uri/*
! gsutil cat $destination_uri/prediction*/*.jsonl


*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015151/batch_output/prediction-flowers_20210226015151-2021-02-26T03:00:47.533913Z/image_classification_0.jsonl
gs://migration-ucaip-trainingaip-20210226015151/batch_output/prediction-flowers_20210226015151-2021-02-26T03:00:47.533913Z/image_classification_1.jsonl
{"ID":"gs://migration-ucaip-trainingaip-20210226015151/18089878729_907ed2c7cd_m.jpg","annotations":[{"annotation_spec_id":"5160231270013206528","classification":{"score":0.9993481},"display_name":"dandelion"}]}
{"ID":"gs://migration-ucaip-trainingaip-20210226015151/754296579_30a9ae018c_n.jpg","annotations":[{"annotation_spec_id":"8907226159985459200","classification":{"score":1},"display_name":"daisy"}]}
```



## Make online predictions

### Prepare file for online prediction

### [projects.locations.models.deploy](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/deploy)

#### Call

In [ ]:
request = clients["automl"].deploy_model(
    name=model_id
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:
```
{}
```



### [projects.locations.models.predict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/predict)

#### Request

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
test_item = test_item[0].split(",")[0]

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

payload = [{
    "image": {
        "image_bytes": content
    }
}]

params = {"score_threshold": "0.8"}

prediction_r = automl.PredictRequest(
    name=model_id,
    payload=payload,
    params=params
)

print(MessageToJson(
    automl.PredictRequest(
        name=model_id,
        payload=payload,
        params=params
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/ICN3600040762873479168",
  "payload": {
    "image": {
      "imageBytes": "..."
    }
  },
  "params": {
    "score_threshold": "0.8"
  }
}
```



#### Call

In [ ]:
request = clients["prediction"].predict(
    name=model_id,
    payload=payload,
    params=params
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "payload": [
    {
      "annotationSpecId": "8907226159985459200",
      "classification": {
        "score": 1.0
      },
      "displayName": "daisy"
    }
  ]
}
```



### [projects.locations.models.undeploy](https://cloud.google.com/automl/docs/reference/rest/v1/projects.locations.models/undeploy)

#### Request

In [ ]:
request = clients["automl"].undeploy_model(
    name=model_id
)


#### Call

In [ ]:
result = request.result()


#### Response



In [ ]:
print(MessageToJson(result))


*Example output*:

```
{}
```



## Train and export an Edge model

### [projects.locations.models.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

#### Request

In [ ]:
# creating edge model for export
model_edge = {
    "display_name": "flowers_edge_" + TIMESTAMP,
    "dataset_id": dataset_short_id,
    "image_classification_model_metadata":{
        "train_budget_milli_node_hours": 8000,
        "model_type": "mobile-versatile-1"
    },
}

print(
    MessageToJson(
        automl.CreateModelRequest(
            parent=PARENT,
            model=model_edge    
        ).__dict__["_pb"]
    )
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "flowers_edge_20210226015151",
    "datasetId": "ICN2833688305139187712",
    "imageClassificationModelMetadata": {
      "modelType": "mobile-versatile-1",
      "trainBudgetMilliNodeHours": "8000"
    }
  }
}
```



#### Call

In [ ]:
request = clients["automl"].create_model(
    parent=PARENT,
    model=model_edge
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/ICN8566948201909714944"
}
```



In [ ]:
model_edge_id = result.name
    

### [projects.locations.models.export](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/export)

In [ ]:
output_config = {
    "model_format": "tflite",
    "gcs_destination": {
        "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/export/",
    }
}


print(MessageToJson(
    automl.ExportModelRequest(
        name=model_edge_id,
        output_config=output_config
    ).__dict__["_pb"])
)


*Example output*:

```
{
  "name": "projects/116273516712/locations/us-central1/models/ICN8566948201909714944",
  "outputConfig": {
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015151/export/"
    },
    "modelFormat": "tflite"
  }
}
```



#### Call

In [ ]:
request = clients["automl"].export_model(
    name=model_edge_id,
    output_config=output_config
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:

```
{}
```



In [ ]:
model_export_dir  = output_config["gcs_destination"]["output_uri_prefix"]

! gsutil ls -r $model_export_dir


*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015151/export/:

gs://migration-ucaip-trainingaip-20210226015151/export/model-export/:

gs://migration-ucaip-trainingaip-20210226015151/export/model-export/icn/:

gs://migration-ucaip-trainingaip-20210226015151/export/model-export/icn/tflite-flowers_edge_20210226015151-2021-02-26T06:16:19.437101Z/:
gs://migration-ucaip-trainingaip-20210226015151/export/model-export/icn/tflite-flowers_edge_20210226015151-2021-02-26T06:16:19.437101Z/dict.txt
gs://migration-ucaip-trainingaip-20210226015151/export/model-export/icn/tflite-flowers_edge_20210226015151-2021-02-26T06:16:19.437101Z/model.tflite
gs://migration-ucaip-trainingaip-20210226015151/export/model-export/icn/tflite-flowers_edge_20210226015151-2021-02-26T06:16:19.437101Z/tflite_metadata.json
```



# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.



In [ ]:
delete_dataset = True
delete_model = True
delete_bucket = True

# Delete the dataset using the AutoML fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['automl'].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the AutoML fully qualified identifier for the model
try:
    if delete_model:
        clients['automl'].delete_model(name=model_id)
except Exception as e:
    print(e)
    
# Delete the model using the AutoML fully qualified identifier for the model
try:
    if delete_model:
        clients['automl'].delete_model(name=model_edge_id)
except Exception as e:
    print(e) 

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
